# Service deployment using TrueFoundry


##### This notebook demonstrates a demo of how you can deploy your first service with TrueFoundry.

---


After you complete the guide, you will have a successfully deployed model. Your deployed API will look like this:

![](https://files.readme.io/e1affc0-Screenshot_2022-11-11_at_5.07.48_PM.png)


## Project structure

To complete this guide, you are going to create the following **files**:

- `app.py` : contains our inference and FastAPI code
- `iris_classifier.joblib` : the model file
- `deploy.py`: contains our deployment code
- `requirements.txt` : contains our dependencies

Your **final file structure** is going to look like this:

```
.
├── app.py
├── iris_classifier.joblib
├── deploy.py
└── requirements.txt
```

As you can see, all the following files are created in the same folder/directory

**Let's create the directory which will contain all this files:-**


In [ ]:
%mkdir develop
%cd develop

# Setup

Let's first setup all the things we need to deploy our service.

- Signup or Login on TrueFoundry
- Setup Workspace


Let's start with installing truefoundry.


In [ ]:
%pip install -U "truefoundry[ml]>=0.2.0,<1.0.0"

**Login into TrueFoundry**

In order to login run the cell below. Host can be found from the TrueFoundry UI as shown below like https://app.truefoundry.com

![image.png](../common/images/host.png)

In [ ]:
!tfy login --host "<Host name of TrueFoundry UI. e.g. https://company.truefoundry.cloud>"

**Select the `Workspace` in which you want to deploy your application.**

Once you run the cell below you will get a prompt to enter your Workspace FQN. Follow the docs to

**Create a Workspace**: https://docs.truefoundry.com/docs/key-concepts#creating-a-workspace  

In [ ]:
WORKSPACE_FQN = input("Enter your Workspace FQN: ")

# Step 1: Fetch model

## Model details

For this guide, we have already trained a model.  
The given model has been trained on _iris dataset_. Then it is stored as a joblib file in [google drive](https://drive.google.com/file/d/1-9nwjs6F7cp_AhAlBAWZHMXG8yb2q_LR/view).

> **Attributes** :  
> sepal length in cm, sepal width in cm, petal length in cm, petal width in cm
>
> **Predicted Attribute** :  
> class of iris plant (one of the following - Iris Setosa, Iris Versicolour, Iris Virginica)

## Download instructions

Download the model from the following [link](https://drive.google.com/file/d/1-9nwjs6F7cp_AhAlBAWZHMXG8yb2q_LR/view).  
Then move the model in your dev directory we created.

Afterwards, your directory should look like this :

```
.
└── iris_classifier.joblib
```


# Step 2: Implement the Inference Service code.

The first step is to create a web API and deploy the model.  
For this we are going to use [FastAPI](https://fastapi.tiangolo.com/) for this. FastAPI is a modern, intuitive web framework for building web APIs in python.

Create the `app.py` and `requirements.txt` files in the same directory where the model is stored.

```
.
├── iris_classifier.joblib
├── app.py
└── requirements.txt
```


### **`app.py`**


In [ ]:
%%writefile app.py
import os
import joblib
import pandas as pd
from fastapi import FastAPI

model = joblib.load("iris_classifier.joblib")

app = FastAPI()

@app.post("/predict")
def predict(
    sepal_length: float, sepal_width: float, petal_length: float, petal_width: float
):
    data = dict(
        sepal_length=sepal_length,
        sepal_width=sepal_width,
        petal_length=petal_length,
        petal_width=petal_width,
    )
    prediction = int(model.predict(pd.DataFrame([data]))[0])
    return {"prediction": prediction}

Click on this [link](https://docs.truefoundry.com/recipes/create-a-fastapi-service-code-to-deploy-model) to understand the **`app.py`**:


In [ ]:
%%writefile requirements.txt
fastapi==0.81.0
uvicorn==0.18.3
scikit-learn==1.2.2
joblib==1.3.2
pandas==2.1.0


# Step 3: Deploying the Inference App

You can deploy services on TrueFoundry programmatically via our **Python SDK**.

Create a `deploy.py`, after which our file structure will look like this:

**File Structure**

```Text Text
.
├── iris_classifier.joblib
├── app.py
├── deploy.py
└── requirements.txt
```

### **`deploy.py`**


In [ ]:
%%writefile deploy.py
import argparse
import logging
from truefoundry.deploy import Build, PythonBuild, Service, Resources, Port, LocalSource

logging.basicConfig(level=logging.INFO)

parser = argparse.ArgumentParser()
parser.add_argument("--name", required=True, type=str, help="Name of the application.")
parser.add_argument(
    "--workspace_fqn",
    required=True,
    type=str,
    help="FQN of the workspace where application will be deployed.",
)
parser.add_argument(
    "--host",
    required=True,
    type=str,
    help="Host where the application will be available for access. Ex:- my-app.my-org.com",
)
args = parser.parse_args()

image = Build(
    build_source=LocalSource(local_build=False),
    build_spec=PythonBuild(
        python_version="3.11",
        command="uvicorn app:app --port 8000 --host 0.0.0.0",
        requirements_path="requirements.txt",
    )
)

service = Service(
    name=args.name,
    image=image,
    ports=[Port(port=8000, host=args.host)],
    resources=Resources(
        cpu_request=0.1,
        cpu_limit=0.1,
        memory_request=500,
        memory_limit=500,
    ),
    env={"UVICORN_WEB_CONCURRENCY": "1", "ENVIRONMENT": "dev"},
)
service.deploy(workspace_fqn=args.workspace_fqn, wait=False)

Click on this [link](https://docs.truefoundry.com/recipes/deploy-fastapi-service-via-python) to understand the **`deploy.py`**:


In [ ]:
SERVICE_NAME = input("Enter the Service name")
SERVICE_HOST = input(
    "Enter the Service Host (Can be found from cluster details in TrueFoundry UI)"
)

Now to deploy our FastAPI Service run the command below


In [ ]:
!python deploy.py --name $SERVICE_NAME --workspace_fqn $WORKSPACE_FQN --host $SERVICE_HOST

Once the build is complete, you will see a link to the dashboard after a message like <br>
`You can find the application on the dashboard:-`.

Click on the link to access the deployment dashboard.
